In [1]:
#NB!https://github.com/estnltk/estnltk/blob/master/README.md 
import urllib.request, os
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import shutil
import os

In [2]:
data = pd.read_csv("Pohitabel_stopid_eemald_lower.csv")
data

,Tag,Lõigu sisu,Pealkiri1,Kohtuasja_nr,Kohtuasja_ID,algus_lopp,stop_eemaldatud
0,C04Titre1,Eelotsuse küsimuse analüüs,NaN,C‑416/20 PPU,0,algus,eelotsuse küsimuse analüüs
1,C01PointnumeroteAltN,"27 Kõigepealt väärib märkimist, et väljak...",NaN,C‑416/20 PPU,0,algus,27 väärib märkimist väljakujunenud kohtuprakti...
2,C01PointnumeroteAltN,28 Käesoleval juhul nähtub eelotsusetaotl...,NaN,C‑416/20 PPU,0,algus,28 käesoleval nähtub eelotsusetaotlusest eelot...
3,C01PointnumeroteAltN,29 Eelotsusetaotluse esitanud kohtu hinna...,NaN,C‑416/20 PPU,0,algus,29 eelotsusetaotluse esitanud kohtu hinnangul ...
4,C01PointnumeroteAltN,30 Neil asjaoludel tuleb mõista eelotsuse...,NaN,C‑416/20 PPU,0,algus,30 asjaoludel mõista eelotsusetaotluse esitanu...
...,...,...,...,...,...,...,...
354266,C01PointnumeroteAltN,95 Kõnesoleva hüvitise hindamisel tuleks ...,NaN,F‑1/05,23801,algus,95 kõnesoleva hüvitise hindamisel arvesse võtt...
354267,C48DispositifIntroduction,Esitatud põhjendustest lähtudes,Resolutsioon,F‑1/05,23801,algus,esitatud põhjendustest lähtudes
354268,C08Dispositif,1. Tühistada Euroopa Koolitusfondi 25. ju...,Resolutsioon,F‑1/05,23801,algus,1. tühistada euroopa koolitusfondi 25. juuni 2...
354269,C08Dispositif,2. Pooled teatavad Avaliku Teenistuse Koh...,Resolutsioon,F‑1/05,23801,algus,2. teatavad avaliku teenistuse kohtule kolme k...


In [3]:
#eemaldame null väärtustega read ja paneme listi
puhas=data[~pd.isnull(data.stop_eemaldatud)]
column_list=puhas['stop_eemaldatud'].to_list()

In [4]:
# https://colab.research.google.com/drive/1Uh-w_oCllFMaUQOBZVL6xsdSyOfXBTxK?usp=sharing#scrollTo=AUORCu_ztPwG 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()

In [5]:
#Meetod teksti sõnestamiseks ja lemmatiseerimiseks estnltk abil
from estnltk import Text
def lemmatize_with_estnltk(input_string):
    lemmas = []
    text = Text(input_string).tag_layer()
    for word in text.morph_analysis:
        lemmas.append(word.lemma[0])
    return lemmas

vectorizer = TfidfVectorizer(tokenizer = lemmatize_with_estnltk)

In [ ]:
# Andmete viimine vektorkujule
matrix = vectorizer.fit_transform(column_list)
matrix.shape

In [92]:
#loodud proovilause, millega testida sarnasusi
listike=['käibemaksukohustuslaseks registreerimine tinkivinki']

In [93]:
# Proovilause viimine vektorkujule
vordluslist = vectorizer.fit_transform(listike)
vordluslist.shape

(1, 5)

In [55]:
from sklearn.metrics.pairwise import cosine_distances

# Leiame kõigi tekstide kaugused üksteisest
cos_dist = cosine_distances(matrix)
cos_dist

array([[0.        , 0.9675383 , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.9675383 , 0.        , 0.93034207, ..., 0.88511062, 0.91100655,
        0.92726082],
       [1.        , 0.93034207, 0.        , ..., 0.92512227, 0.90631686,
        0.97114606],
       ...,
       [1.        , 0.88511062, 0.92512227, ..., 0.        , 0.83421171,
        0.95547117],
       [1.        , 0.91100655, 0.90631686, ..., 0.83421171, 0.        ,
        0.90277778],
       [1.        , 0.92726082, 0.97114606, ..., 0.95547117, 0.90277778,
        0.        ]])

In [57]:
# Uurime tekste, mis on omavahel mõnevõrra sarnased
texts=column_list_remove_stop[0:100]
for idx, k in enumerate(cos_dist):
    for idx2, j in enumerate(k):
        if idx2 > idx:
            if j > 0.8 and j < 1:
                print(idx, idx2)
                print(j, '\n\n', texts[idx], '\n\n', texts[idx2], '\n', '-'*40, '\n')

0 1
0.9675383041901787 

 eelotsuse küsimuse analüüs 

 27 väärib märkimist, väljakujunenud kohtupraktika eltl artiklis 267 sätestatud liikmesriigi kohtute euroopa kohtu vahelises koostöömenetluses euroopa kohtu ülesanne anda liikmesriigi kohtule tarvilik vastus, võimaldaks viimasel poolelioleva kohtuasja lahendada. arvestades euroopa kohtul esitatud küsimused vajaduse sõnastada (2. aprilli 2020. aasta kohtuotsus ruska federacija, c‑897/19 ppu, eu:c:2020:262, punkt 43 viidatud kohtupraktika). 
 ---------------------------------------- 

0 4
0.9308415089244562 

 eelotsuse küsimuse analüüs 

 30 asjaoludel mõista eelotsusetaotluse esitanud kohtu küsimust nii, puudutab küsimust, raamotsuse 2002/584 artiklit 4a tõlgendada nii, vahistamismäärust täitev õigusasutus keelduda euroopa vahistamismääruse täitmisest, vabadusekaotuse vabadust piirava julgeolekumeetme täitmiseks, asjaomane isik takistanud kohtukutse isiklikult kättetoimetamist ilmunud isiklikult kohtulikule arutelule, põgenenud vah

 28 käesoleval nähtub eelotsusetaotlusest, eelotsusetaotluse esitanud kohus otsuse kohta, tri üleandmine rumeenia ametiasutustele õiguspärane seaduse rahvusvahelise õigusabi kriminaalasjades § 83 sätete alusel, rakendatakse raamotsuse 2002/584 artiklit 4a saksa õiguses. 

 58 kolmanda küsimusega, koosneb kahest osast, soovib eelotsusetaotluse esitanud kohus sisuliselt teada esiteks, juhul, euroopa vahistamismääruse liikmesriigi õigusaktides puuduvad sätted, näeksid kohtusse pöördumise võimaluse, kontrollida, tingimustel euroopa vahistamismääruse asutus, osaleb liikmesriigi õigusemõistmises, kohus, raamotsust 2002/584 koostoimes harta artikliga 47 tagatud õigusega tõhusale kohtulikule kaitsele tõlgendada nii, lubab liikmesriigi kohtul, pöördutud isiku vahi pidamise õiguspärasuse vaidlustamiseks, antud euroopa vahistamismääruse täitmisel, riigisisese akti alusel, „[riigisiseseks] vahistamismäärus[eks] tahes muu[ks] mõjuga täidetav[aks] kohtuotsus[eks]“ raamotsuse artikli 8 lõike 1 punkti

 ---------------------------------------- 

5 35
0.874207165120558 

 31 seoses märkida, nähtub raamotsuse 2002/584 artikli 1 lõigetest 1 2 põhjendustest 5 7, raamotsuse eesmärk asendada liikmesriikidevaheline mitmepoolne väljaandmissüsteem, põhineb 13. detsembril 1957 pariisis allkirjastatud väljaandmise euroopa konventsioonil, vastastikuse tunnustamise põhimõttel rajaneva õigusasutustevahelise üleandmissüsteemiga süüdimõistetute kriminaalkaristuste täitmiseks kahtlustatavate kriminaalmenetluse läbiviimiseks (29. jaanuari 2013. aasta kohtuotsus radu, c‑396/11, eu:c:2013:39, punkt 33, 11. märtsi 2020. aasta kohtuotsus sf (euroopa vahistamismäärus – vahistamismäärust täitvasse riiki tagasitoomise tagatis), c‑314/18, eu:c:2020:191, punkt 37 viidatud kohtupraktika). 

 41 esimese küsimusega palub eelotsusetaotluse esitanud kohus sisuliselt selgitada, raamotsuse 2002/584 artikli 6 lõiget 1 tõlgendada nii, „vahistamismääruse õigusasutuseks“ pidamine sätte tähenduses sõltub sellest, euroopa 


8 17
0.9051163800803121 

 34 järelikult, euroopa vahistamismääruse täitmine reegel, kujutab täitmisest keeldumine erandit, tõlgendada kitsalt (25. juuli 2018. aasta kohtuotsus minister for justice and equality (kohtusüsteemi puudused), c‑216/18 ppu, eu:c:2018:586, punkt 41 viidatud kohtupraktika). 

 43 puutub direktiivi 2016/343, eelotsusetaotluse esitanud kohus viitab, märkida, direktiivi artikli 8 lõige 1 annab kahtlustatavatele süüdistatavatele õiguse viibida kohtulikul arutelul. artikli lõike 2 liikmesriigid näha ette, kohtulik arutelu, tulemusena otsustatakse kahtlustatava süüdistatava süü süü puudumise üle, toimuda kohalviibimiseta, lõikes loetletud tingimused täidetud. 
 ---------------------------------------- 

8 18
0.9156539942936315 

 34 järelikult, euroopa vahistamismääruse täitmine reegel, kujutab täitmisest keeldumine erandit, tõlgendada kitsalt (25. juuli 2018. aasta kohtuotsus minister for justice and equality (kohtusüsteemi puudused), c‑216/18 ppu, eu:c:2018:586, p

12 20
0.8929083351502014 

 38 tuleneb raamotsuse 2002/584 artikli 4a lõike 1 sõnastusest, vahistamismäärust täitev õigusasutus keelduda täitmast euroopa vahistamismäärust, vabadusekaotuse vabadust piirava julgeolekumeetme täitmiseks, isik ilmunud isiklikult kohtulikule arutelule, tulemusel otsus tehti, arvatud juhul, euroopa vahistamismääruses märgitakse, sätte punktides a–d sätestatud tingimused täidetud (10. augusti 2017. aasta kohtuotsus tupikas, c‑270/17 ppu, eu:c:2017:628, punkt 54 viidatud kohtupraktika) 

 46 kontekstis vahistamismääruse liikmesriigi õiguse võimalik vastuolu direktiivi 2016/343 sätetega põhjus, viia euroopa vahistamismääruse täitmisest keeldumiseni. 
 ---------------------------------------- 

12 23
0.8271108996134768 

 38 tuleneb raamotsuse 2002/584 artikli 4a lõike 1 sõnastusest, vahistamismäärust täitev õigusasutus keelduda täitmast euroopa vahistamismäärust, vabadusekaotuse vabadust piirava julgeolekumeetme täitmiseks, isik ilmunud isiklikult kohtulikule a

...

71 99
0.9227626906388912 

 74 kolmanda küsimuse esimesele osale vastata, juhul, euroopa vahistamismääruse liikmesriigi õigusaktides puuduvad sätted, näeksid kohtusse pöördumise võimaluse, kontrollida, tingimustel euroopa vahistamismääruse asutus, osaleb liikmesriigi õigusemõistmises, kohus, raamotsust 2002/584 koostoimes harta artikliga 47 tagatud õigusega tõhusale kohtulikule kaitsele tõlgendada nii, lubab liikmesriigi kohtul, pöördutud isiku vahi pidamise õiguspärasuse vaidlustamiseks, antud euroopa vahistamismääruse täitmisel, riigisisese akti alusel, „[riigisiseseks] vahistamismäärus[eks] tahes muu[ks] mõjuga täidetav[aks] kohtuotsus[eks]“ raamotsuse artikli 8 lõike 1 punkti tähenduses, menetluses esitatud väide, euroopa vahistamismäärus liidu õiguse seisukohast kehtetu, tunnistada pädevaks kehtivust kontrollima. 

 42 puuduste esinemine mõjuta tingimata lahendit, liikmesriigi kohtud üksikjuhul teha. 
 ---------------------------------------- 

72 73
0.8631838595532992 

 euroopa 

 76 nähtuvalt raamotsuse 2002/584 artikli 1 lõikest 1 euroopa vahistamismääruse mehhanismi eesmärk võimaldada tagaotsitav kinni anda, – raamotsuse eesmärki silmas – toimepandud kuritegu jääks karistamata isiku viia kriminaalmenetluse kannaks mõistetud vabadusekaotusliku karistuse (6. detsembri 2018. aasta kohtuotsus ik (lisakaristuse täideviimine), c‑551/18 ppu, eu:c:2018:991, punkt 39). 

 34 esitatud küsimustele vastamiseks esiteks kindlaks teha, raamotsuse 2002/584 artikli 6 lõiget 1 tõlgendada nii, vahistamismäärust täitev õigusasutus keelduda tunnustamast euroopa vahistamismääruse kohut „vahistamismääruse õigusasutusena“ sätte tähenduses ainuüksi põhjusel, teavet vahistamismääruse liikmesriigi kohtuvõimu sõltumatusega seotud süsteemsete üldiste puuduste kohta, esinesid vahistamismääruse tegemise ajal tekkisid vahistamismääruse tegemist. 
 ---------------------------------------- 

74 97
0.8964001996700796 

 76 nähtuvalt raamotsuse 2002/584 artikli 1 lõikest 1 euroopa vahistamismä

 41 vahistamismäärust täitev õigusasutus, teavet vahistamismääruse liikmesriigi kohtuvõimu sõltumatusega seotud süsteemsete üldiste puuduste kohta, esinesid vahistamismääruse tegemise ajal tekkisid vahistamismääruse tegemist, keelduda tunnustamast „vahistamismääruse õigusasutusena“ raamotsuse 2002/584 artikli 6 lõike 1 tähenduses liikmesriigi kohtunikke kohtuid, olemuselt tegutsevad sõltumatult täitevvõimust. 
 ---------------------------------------- 

77 99
0.9589143677535038 

 79 lisaks, ühtlustatud tingimusi, kriminaalmenetluses isiku tõkendina vahistamise kohaldamine otsustada vahistus jõusse jätta (vt 19. september 2018. aasta kohtuotsus milev, c‑310/18 ppu, eu:c:2018:732, punkt 47, 28. novembri 2019. aasta kohtuotsus spetsializirana prokuratura, c‑653/19 ppu, eu:c:2019:1024, punkt 28), pädev kohus riigisiseses õiguses nähtud tingimustel tõkendi otsustada vajaduse kohaldamise peatada, tuvastab, tingimused täidetud. 

 42 puuduste esinemine mõjuta tingimata lahendit, liikmesriigi

 83 kaalutlustest tulenevalt kolmandale küsimusele vastata järgmiselt: 

 38 eeldab vastastikuse tunnustamise põhimõte, euroopa vahistamismäärused raamotsuse 2002/584 artikli 1 lõike 1 tähenduses kuuluvad raamotsuse sätete alusel täitmisele, eeldab, euroopa vahistamismääruse, sättes määratletud „kohtuotsus“, „õigusasutus“ raamotsuse artikli 6 lõike 1 tähenduses (27. mai 2019. aasta kohtuotsus og pi (lübecki prokuratuur zwickau prokuratuur), c‑508/18 c‑82/19 ppu, eu:c:2019:456, punkt 46 viidatud kohtupraktika). viimati nimetatud mõiste tähendab, asjaomane asutus tegutseb euroopa vahistamismääruse tegemisega seotud ülesannete täitmisel sõltumatult (vt 27. mai 2019. aasta kohtuotsus og pi (lübecki prokuratuur zwickau prokuratuur), c‑508/18 c‑82/19 ppu, eu:c:2019:456, punktid 74 88). 
 ---------------------------------------- 

81 96
0.9711272223467077 

 83 kaalutlustest tulenevalt kolmandale küsimusele vastata järgmiselt: 

 39 oluline rõhutada, kohtute sõltumatuse nõue osa õiguse õiglas

 ---------------------------------------- 

94 97
0.8833218640381573 

 37 tuleneb, vahistamismäärust täitvad õigusasutused põhimõtteliselt keelduda määruse täitmisest ammendavalt loetletud alustel, nähtud raamotsuses 2002/584, euroopa vahistamismääruse täitmisele seada üksnes raamotsuse artiklis 5 ammendavalt nähtud tingimuse. järelikult, euroopa vahistamismääruse täitmine kujutab reeglit, täitmisest keeldumine erand, tõlgendada kitsalt (25. juuli 2018. aasta kohtuotsus minister for justice and equality (kohtusüsteemi puudused), c‑216/18 ppu, eu:c:2018:586, punkt 41 viidatud kohtupraktika). 

 40 asjaolusid arvestades liikmesriik selleks, tagada raamotsusega 2002/584 kehtestatud euroopa vahistamismääruse mehhanismi toimimise aluseks olevate vastastikuse usalduse vastastikuse tunnustamise põhimõtete täielik kohaldamine, euroopa kohtu lõppkontrolli tagama kohtuvõimu sõltumatuse säilimise, hoidudes meetmetest, kahjustada. 
 ---------------------------------------- 

94 99
0.9344550811693

In [ ]:
# vaja oleks siia veel võrdlusi teha!